<a href="https://colab.research.google.com/github/jnam4/BIA660-Web-Mining-Projects/blob/main/Movie_Scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Project Part I
## Group 3: Emily Su, Jiyun Nam, Ruitong Sun

In [ ]:
!apt-get update
!apt install chromium-chromedriver -y
!pip install selenium
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install webdriver-manager

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,659 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,718 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jamm

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument("--disable-extensions")
options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 180)


import requests
from bs4 import BeautifulSoup
import json


In [ ]:
driver.get("https://www.rottentomatoes.com/browse/movies_at_home/sort:popular")
time.sleep(1)


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# List of guide URLs
urls = [
    "https://editorial.rottentomatoes.com/guide/worst-movies-of-all-time/",
    "https://editorial.rottentomatoes.com/guide/best-2024-movies-every-certified-fresh/",
    "https://editorial.rottentomatoes.com/guide/best-movies-of-2023/",
    "https://editorial.rottentomatoes.com/guide/best-movies-2022/",
    "https://editorial.rottentomatoes.com/guide/2021-best-movies/",
    "https://editorial.rottentomatoes.com/guide/best-netflix-shows-and-movies-to-binge-watch-now/",
    "https://editorial.rottentomatoes.com/guide/best-netflix-movies-to-watch-right-now/"
]

movie_data = []

for url in urls:
    print(f"Scraping: {url}")
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    movie_divs = soup.find_all('div', class_='col-sm-6 col-full-xs')

    for div in movie_divs:
        a_tag = div.find('a', class_='article_movie_poster')
        if a_tag and 'href' in a_tag.attrs:
            link = a_tag['href']
            full_link = "https://www.rottentomatoes.com" + link if link.startswith('/') else link
            title_div = div.find_next_sibling('div', class_='col-sm-18 col-full-xs countdown-item-content')
            if title_div:
                h2_tag = title_div.find('h2')
                if h2_tag:
                    movie_name_tag = h2_tag.find('a')
                    if movie_name_tag:
                        movie_name = movie_name_tag.text.strip()
                        movie_data.append((movie_name, full_link))

df = pd.DataFrame(movie_data, columns=["Movie_Name", "Movie_Link"])
df = df.drop_duplicates(subset="Movie_Name", keep="first")
df.to_csv("movie_links_no_duplicates.csv", index=False)

print(f"\nTotal unique movies saved: {len(df)}")
print("Saved to movie_links_no_duplicates.csv")


Scraping: https://editorial.rottentomatoes.com/guide/worst-movies-of-all-time/
Scraping: https://editorial.rottentomatoes.com/guide/best-2024-movies-every-certified-fresh/
Scraping: https://editorial.rottentomatoes.com/guide/best-movies-of-2023/
Scraping: https://editorial.rottentomatoes.com/guide/best-movies-2022/
Scraping: https://editorial.rottentomatoes.com/guide/2021-best-movies/

Total unique movies saved: 824
Saved to movie_links_no_duplicates.csv


In [ ]:
def scrape_movie_info(urls):
    test_limit = 3
    limited_urls = urls[:test_limit]

    movie_data = []

    for i, url in enumerate(limited_urls):
        print(f"Scraping ({i + 1}/{len(limited_urls)}): {url}")
        try:
            driver.get(url)
            wait.until(EC.presence_of_element_located((By.TAG_NAME, 'h1')))
        except Exception as e:
            print(f"Page load error for {url}: {e}")
            continue

        try:
            title = driver.find_element(By.TAG_NAME, "h1").text
        except:
            title = "N/A"

        try:
            audience_score = driver.execute_script("""
                const el = document.querySelector('rt-text[slot="audienceScore"]');
                return el ? el.innerText : 'N/A';
            """)
        except:
            audience_score = "N/A"
        soup = BeautifulSoup(driver.page_source, "html.parser")
        media_info = soup.find("section", {"data-qa": "section:media-info"})

        def get_item_value(label):
            try:
                block = media_info.find("rt-text", string=label).find_parent("div", class_="category-wrap")
                values = block.find_all(attrs={"data-qa": "item-value"})
                return ", ".join([v.get_text(strip=True) for v in values])
            except:
                return "N/A"

        director = get_item_value("Director")
        rating = get_item_value("Rating")
        language = get_item_value("Original Language")
        box_office = get_item_value("Box Office (Gross USA)")
        runtime = get_item_value("Runtime")
        genre = get_item_value("Genre")
        distributor = get_item_value("Distributor")
        production_co = get_item_value("Production Co")

        try:
            critics_reviews = soup.find("rt-link", {"slot": "collapsedCriticsLink"})
            critics_reviews = critics_reviews.get_text(strip=True) if critics_reviews else "N/A"
        except:
            critics_reviews = "N/A"

        try:
            critics_score = soup.find("rt-text", {"slot": "collapsedCriticsScore"})
            critics_score = critics_score.get_text(strip=True) if critics_score else "N/A"
        except:
            critics_score = "N/A"

        try:
            audience_score_collapsed = soup.find("rt-text", {"slot": "collapsedAudienceScore"})
            audience_score_collapsed = audience_score_collapsed.get_text(strip=True) if audience_score_collapsed else "N/A"
        except:
            audience_score_collapsed = "N/A"

        try:
            verified_ratings = soup.find("rt-link", {"slot": "collapsedAudienceLink"})
            verified_ratings = verified_ratings.get_text(strip=True) if verified_ratings else "N/A"
        except:
            verified_ratings = "N/A"

        try:
            critics_consensus = soup.select_one("div#critics-consensus > p")
            critics_consensus = critics_consensus.get_text(strip=True) if critics_consensus else "N/A"
        except:
            critics_consensus = "N/A"

        try:
            audience_says = soup.select_one("div#audience-consensus > p")
            audience_says = audience_says.get_text(strip=True) if audience_says else "N/A"
        except:
            audience_says = "N/A"

        movie_data.append({
            "Title": title,
            "Audience Score": audience_score,
            "Collapsed Audience Score": audience_score_collapsed,
            "Critics Score": critics_score,
            "Critics Reviews": critics_reviews,
            "Verified Ratings": verified_ratings,
            "Director": director,
            "Rating": rating,
            "Language": language,
            "Box Office": box_office,
            "Runtime": runtime,
            "Genre": genre,
            "Distributor": distributor,
            "Production Co": production_co,
            "Critics Consensus": critics_consensus,
            "Audience Says": audience_says,
            "URL": url
        })

    return pd.DataFrame(movie_data)


import pandas as pd

movie_urls_df = df

movie_urls = movie_urls_df['Movie_Link'].tolist()

df_movies = scrape_movie_info(movie_urls)

print(df_movies)

df_movies.to_csv("rottentomatoes_movies_scraped.csv", index=False)



Scraping (1/3): https://www.rottentomatoes.com/m/ballistic_ecks_vs_sever
Scraping (2/3): https://www.rottentomatoes.com/m/one_missed_call
Scraping (3/3): https://www.rottentomatoes.com/m/left_behind_2014
                       Title Audience Score Collapsed Audience Score  \
0  Ballistic: Ecks vs. Sever            21%                      21%   
1            One Missed Call            29%                      29%   
2                Left Behind            20%                      20%   

  Critics Score Critics Reviews Verified Ratings           Director  \
0            0%     118 Reviews  10,000+ Ratings  Wych Kaosayananda   
1            0%      79 Reviews  50,000+ Ratings       Éric Valette   
2            0%      69 Reviews  10,000+ Ratings      Vic Armstrong   

                                              Rating Language Box Office  \
0                                R (Strong Violence)  English     $14.3M   
1  PG-13 (Intense Sequences of Terror|Frightening...  English     $26.